In [2]:
#import relevant libraries

import requests
import json
from pprint import pprint
import operator
import numpy
import pandas 

# from datetime module
from datetime import date
from datetime import timedelta
from pathlib import Path  

# My Team info
team_id = 3402291


# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
#import graphviz


/var/folders/sz/1kc6hj850ps8d9bbrtzsswcw0000gn/T/ipykernel_25781/3232028448.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


### Overall Player Transfer Data

In [3]:
# Request Information on all players today (now?)
base_url = 'https://fantasy.premierleague.com/api/'
# with open('/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/pipeline/Saved_Data/2024-02-04/2024-02-04.json') as f:
#     r_all_players_today = json.load(f)
r_all_players_today = requests.get(base_url+'bootstrap-static/').json()
pprint(r_all_players_today, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10643865}


In [5]:
# # Get today's date
# today = date.today()

# today_filepath = Path('/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/pipeline/Saved_Data/{}'.format(today))
# today_filepath.mkdir(parents=True, exist_ok=True)

# with open("/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/pipeline/Saved_Data/{}/{}.json".format(today,today), "w+") as f:
#     json.dump(r_all_players_today, f)

In [263]:
overall_events_data = pandas.DataFrame(r_all_players_today['events'])
todays_player_data = pandas.DataFrame(r_all_players_today['elements'])
total_players = r_all_players_today['total_players']

In [264]:
# Total Active players in the game at this point in time (estimate)
current_gameweek = numpy.where(overall_events_data.is_current)[0][0] + 1
total_active_players_estimate = int(round(total_players*(0.97)**current_gameweek,))

In [265]:
# Net transfers of players at this moment 

all_players_transfers_in = todays_player_data.transfers_in
all_players_transfers_out = todays_player_data.transfers_out
net_transfers_overall_today = all_players_transfers_in - all_players_transfers_out

In [266]:
# Price Change so far this event 

all_players_price_change_for_current_event = todays_player_data.cost_change_event

In [267]:
# Players prices today

all_players_prices_today = todays_player_data.now_cost

In [268]:
# Player currently flagged red

list_of_all_players_status = todays_player_data.status
boolean_list_of_players_injured = [True if x == 'i' else False for x in list_of_all_players_status]

In [269]:
# Create DataFrame from dictionary of all this transfer data

price_change_dict = {
    "price_change_this_night": numpy.nan, #tomorrow, we define this as today_data.player_prices_today - yesterday_data.player_prices_today
    "net_transfers_in_out_since_last_price_change": 0, #tomorrow, we define this as yesterday_data.net_transfers_in_out_since_last_price_change + today_data_net_transfers_in_out_since_yesterday, unless price change occurs, then we reset to ="Net Transfers In/Out since yesterday
    "net_transfers_in_out_this_day": 0, #tomorrow, we define this as today_data.net_transfers_in_out_overall_as_of_today - yesterday_data.net_transfers_in_out_overall_as_of_today
    "price_change_so_far_for_this_event ": all_players_price_change_for_current_event,
    "total_active_players_estimate": total_active_players_estimate,
    "players_injured": boolean_list_of_players_injured,
    "player_prices_today" : all_players_prices_today,
    "net_transfers_in_out_overall_as_of_today" : net_transfers_overall_today, #not to be used in modelling
    "player_ids" : todays_player_data.id,
    "player_name": todays_player_data.web_name
}

# Dictionary into DataFrame
today_data = pandas.DataFrame(price_change_dict)
#today_data.set_index("player_ids", inplace = True)


# Player transfer data is updated at least every 40 minutes, and probably every 30 or even every 15 minutes
# This dataframe is a snapshot at 8pm GMT Jan 29th

In [270]:
today_data

,price_change_this_night,net_transfers_in_out_since_last_price_change,net_transfers_in_out_this_day,price_change_so_far_for_this_event,total_active_players_estimate,players_injured,player_prices_today,net_transfers_in_out_overall_as_of_today,player_ids,player_name
0,NaN,0,0,0,5276526,False,44,-52636,1,Balogun
1,NaN,0,0,0,5276526,False,39,-8283,2,Cédric
2,NaN,0,0,0,5276526,False,44,-6452,3,M.Elneny
3,NaN,0,0,0,5276526,False,54,-7212,4,Fábio Vieira
4,NaN,0,0,0,5276526,False,50,-1029813,5,Gabriel
...,...,...,...,...,...,...,...,...,...,...
804,NaN,0,0,0,5276526,False,45,26,769,Hesketh
805,NaN,0,0,0,5276526,False,45,166,770,Barnett
806,NaN,0,0,0,5276526,False,45,44,787,Griffiths
807,NaN,0,0,0,5276526,False,40,13,788,Mosquera


In [271]:
# Get today's date
today = date.today() - timedelta(days = 3) 
 
# Get yesterday date
yesterday = today - timedelta(days = 1) 

print("Today is:", today)
print("Yesterday was:", yesterday)

Today is: 2024-02-04
Yesterday was: 2024-02-03


In [318]:
def update_yesterday_data_rows(yesterday_df, today_df):
    # compare the ids from both datasets -> where a new player appears today, make sure you add that row to yesterday with nulls filling it
    #                                    -> where a player is removed today, make sure you remove that row yesterday 
    
    # Get the set of row identifiers from yesterday and today
    yesterday_ids = set(yesterday_df.player_ids)
    today_ids = set(today_df.player_ids)

    # Find new rows added today
    new_rows = today_ids - yesterday_ids

    # Add new rows with null values to yesterday's dataframe
    if new_rows:
        new_rows_data = today_df[today_df.player_ids.isin(new_rows)]
        yesterday_df = pandas.concat([yesterday_df, new_rows_data], ignore_index=True)

    # Find rows removed today
    removed_rows = yesterday_ids - today_ids

    # Remove rows from yesterday's dataframe
    if removed_rows:
        yesterday_df = yesterday_df[~yesterday_df.player_ids.isin(removed_rows)]
        
    # rename index
    today_df.index.rename('ids', inplace=True)
    yesterday_df.index.rename('ids', inplace=True)

    # sort values
    yesterday_df = yesterday_df.sort_values(by='player_ids')
    today_df = today_df.sort_values(by='player_ids')

    # reset indexes
    today_df.set_index('player_ids', inplace=True, drop=False)
    yesterday_df.set_index('player_ids', inplace=True, drop=False)

    return yesterday_df, today_df


In [319]:
#update yesterday's dataframe with today's results 

#get yesterday's saved results 
yesterday_filepath = Path('/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/pipeline/Saved_Data/{}/{}.csv'.format(yesterday,yesterday))
yesterday_data = pandas.read_csv(yesterday_filepath)

# #ensure rows align, accounting for players being added/removed overnight
yesterday_data, today_data = update_yesterday_data_rows(yesterday_data, today_data)

# #updates 
yesterday_data.price_change_this_night = today_data.player_prices_today - yesterday_data.player_prices_today
today_data.net_transfers_in_out_this_day = today_data.net_transfers_in_out_overall_as_of_today - yesterday_data.net_transfers_in_out_overall_as_of_today
today_data.net_transfers_in_out_since_last_price_change = yesterday_data.net_transfers_in_out_since_last_price_change

# Update net transfer since last price change in today's data 
yesterday_data.net_transfers_in_out_since_last_price_change = yesterday_data.net_transfers_in_out_since_last_price_change + yesterday_data.net_transfers_in_out_this_day

#reset rule if price changed overnight 
boolean_player_changes = numpy.squeeze(yesterday_data.price_change_this_night != 0)
today_data.net_transfers_in_out_since_last_price_change = [today_data.net_transfers_in_out_this_day[i] if boolean_player_changes[i] else today_data.net_transfers_in_out_since_last_price_change[i] for i in today_data.net_transfers_in_out_since_last_price_change.index]

# #rewrite yesterday's file 
# yesterday_data.to_csv(yesterday_filepath) 

In [320]:
today_data

,price_change_this_night,net_transfers_in_out_since_last_price_change,net_transfers_in_out_this_day,price_change_so_far_for_this_event,total_active_players_estimate,players_injured,player_prices_today,net_transfers_in_out_overall_as_of_today,player_ids,player_name
player_ids,,,,,,,,,,
1,NaN,0,-25,0,5276526,False,44,-52636,1,Balogun
2,NaN,0,-65,0,5276526,False,39,-8283,2,Cédric
3,NaN,0,-16,0,5276526,False,44,-6452,3,M.Elneny
4,NaN,0,-11,0,5276526,False,54,-7212,4,Fábio Vieira
5,NaN,0,5428,0,5276526,False,50,-1029813,5,Gabriel
...,...,...,...,...,...,...,...,...,...,...
806,NaN,0,8,0,5276526,False,40,36,806,Estève
807,NaN,0,35,0,5276526,False,50,369,807,Wharton
808,NaN,0,62,0,5276526,False,45,214,808,Sels


In [321]:
yesterday_data

,price_change_this_night,net_transfers_in_out_since_last_price_change,net_transfers_in_out_this_day,price_change_so_far_for_this_event,total_active_players_estimate,players_injured,player_prices_today,net_transfers_in_out_overall_as_of_today,player_ids,player_name
player_ids,,,,,,,,,,
1,0,0,0,0,5273881,False,44,-52611,1,Balogun
2,0,0,0,0,5273881,False,39,-8218,2,Cédric
3,0,0,0,0,5273881,False,44,-6436,3,M.Elneny
4,0,0,0,0,5273881,False,54,-7201,4,Fábio Vieira
5,0,0,0,0,5273881,False,50,-1035241,5,Gabriel
...,...,...,...,...,...,...,...,...,...,...
806,0,0,0,0,5273881,False,40,28,806,Estève
807,0,0,0,0,5273881,False,50,334,807,Wharton
808,0,0,0,0,5273881,False,45,152,808,Sels


In [309]:
today_data.net_transfers_in_out_since_last_price_change[610:613]

player_ids
611    0
613    0
614    0
Name: net_transfers_in_out_since_last_price_change, dtype: int64

In [310]:
boolean_player_changes[610:613]

player_ids
611    False
613    False
614    False
Name: price_change_this_night, dtype: bool

In [312]:
today_data.net_transfers_in_out_since_last_price_change.index

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       801, 802, 803, 804, 805, 806, 807, 808, 809, 810],
      dtype='int64', name='player_ids', length=809)

In [314]:
test = [today_data.net_transfers_in_out_this_day[i] if boolean_player_changes[i] else today_data.net_transfers_in_out_since_last_price_change[i] for i in today_data.net_transfers_in_out_since_last_price_change.index]
test


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3171,
 -3643,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 105,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -9785,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [261]:
#save today's dataframe to today's folder

today_filepath = Path('/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/pipeline/Saved_Data/{}'.format(today))
today_filepath.mkdir(parents=True, exist_ok=True)
today_data.to_csv('{}/{}.csv'.format(today_filepath,today), index=False) 

# with open("/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/pipeline/Saved_Data/{}/{}.json".format(today,today), "w+") as f:
#     json.dump(r_all_players_today, f)

In [249]:
boolean_player_changes = numpy.squeeze(yesterday_data.price_change_this_night != 0)
boolean_player_changes

player_ids
1      False
2      False
3      False
4      False
5      False
       ...  
806    False
807    False
808    False
809    False
810    False
Name: price_change_this_night, Length: 809, dtype: bool

In [231]:
today_data

,price_change_this_night,net_transfers_in_out_since_last_price_change,net_transfers_in_out_this_day,price_change_so_far_for_this_event,total_active_players_estimate,players_injured,player_prices_today,net_transfers_in_out_overall_as_of_today,player_ids,player_name
0,NaN,0,0,0,5281103,False,44,-52695,1,Balogun
1,NaN,0,0,0,5281103,False,39,-8415,2,Cédric
2,NaN,0,0,0,5281103,False,44,-6458,3,M.Elneny
3,NaN,0,0,0,5281103,False,54,-7231,4,Fábio Vieira
4,NaN,0,0,0,5281103,False,50,-1011903,5,Gabriel
...,...,...,...,...,...,...,...,...,...,...
804,NaN,0,0,0,5281103,False,45,24,769,Hesketh
805,NaN,0,0,0,5281103,False,45,166,770,Barnett
806,NaN,0,0,0,5281103,False,45,56,787,Griffiths
807,NaN,0,0,0,5281103,False,40,13,788,Mosquera


### Prelim Modelling

In [ ]:
# Read in yesterday's data, which should be a completed dataset if preprocessing has worked
yesterday_data = pandas.read_csv(yesterday_filepath,index_col=['player_ids'])

# Split the data into features (X) and target (y)
X = yesterday_data.drop('price_change_this_night', axis=1)
y = bank_data['price_change_this_night']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Fit a rudimentary RF Classifier 
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make Predictions for Test Set
y_pred = rf.predict(X_test)

# Evaluate Predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

### Personal Team Data

In [ ]:
# Request Information on my team

r_my_team = requests.get(base_url+'entry/{}/event/{}/picks/'.format(team_id,current_gameweek)).json()
my_players_ids = list(map(operator.itemgetter('element'), r_my_team['picks']))
my_players_ids

# Get list of all player IDs
total_list_of_players_ids = list(map(operator.itemgetter('id'), r_all_players_today['elements']))

# Makes a list of all players that is TRUE is player is in my team
boolean_list_of_players_in_my_team = [player in my_players_ids for player in total_list_of_players_ids]

# Use boolean list to exact all info about my players this week
my_players_total = [item for item, condition in zip(r_all_players_today['elements'], boolean_list_of_players_in_my_team) if condition]
my_players_total

# Print all my player's names
for players in my_players_total:
    print(players.get('web_name'))